This notebook shows how to time and plot a simple function of one variable using C++ for the function, and Python for the plotting. To run the notebook, the simplest thing to do is just click the "Cell" menu and select "Run all". Or you can click the "fast forward" icon on the toolbar.

These first lines below are just some Python tricks and package imports, you can ignore them.

In [ ]:
%load_ext cython
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

This is the key cell for you. The first line simply instructs the notebook to write the output into a file called ``timing-code.cpp``. What follows is the code that is written into that file.

There are a couple of things to note about this C++ file. The entry point will be the ``run_timing()`` function. It loops over a few values of ``n``, calculates the time to run the function ``f(n)`` (more on this below), and then writes the timings into a file ``timing-results.txt``.

The function ``busy_sleep(n)`` will wait for ``n`` microseconds. We'll use this to simulate long running code.

The timing code works by repeatedly running ``f(n)`` until at least 100ms have elapsed, and divides the total time taken by the number of times the loop ran. This is important for estimating timings of functions which take a very small amount of time to run.

In [ ]:
%%writefile timing-code.cpp

#include <chrono>
#include <iostream>
#include <fstream>

using namespace std;

// this function busy sleeps for n microseconds
void busy_sleep(int n)
{
    auto start = chrono::steady_clock::now();
    while(chrono::duration_cast<chrono::microseconds>(chrono::steady_clock::now() - start).count()<n) {};
}

///////// this is the function we'll be timing ///////////////////
void f(int n)
{
   for(int i=0; i<n; i++)
   {
      for(int j=0; j<n; j++)
      {
        busy_sleep(1);
      }
   }
}

void run_timing()
{
    ofstream outfile;
    outfile.open("timing-results.txt", ios::out);
    for(int n=0; n<=100; n+=10)
    {
        auto start = chrono::steady_clock::now(); 
        int count = 0;
        while(chrono::duration_cast<chrono::milliseconds>(chrono::steady_clock::now() - start).count()<100) {
            f(n);
            count += 1;
        }
        auto stop = chrono::steady_clock::now(); 
        auto duration = chrono::duration_cast<chrono::microseconds>(stop - start);
        double t = (double)duration.count()/count;
        outfile << n << " " << t << endl;
    }
}

The code below compiles the file defined above using the Cython package.

In [ ]:
%%cython -I . -f --cplus

cdef extern from "timing-code.cpp":
   cpdef void run_timing()

The code below runs the compiled, loads the data into Python and plots the output.

In [ ]:
run_timing()

N, times = np.loadtxt('timing-results.txt').T

plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.plot(N, times)
plt.xlabel('n')
plt.ylabel('Time (us)')
plt.subplot(122)
plt.loglog(N, times);
plt.xlabel('n')
plt.ylabel('Time (us)')
plt.tight_layout()